# YouBot Project Log - December 13, 2025

## 1. Project Overview

**Robot:** KUKA YouBot - Mobile manipulator with:
- Omnidirectional base (4 mecanum wheels)
- 5-DOF arm
- 2-finger gripper
- Hokuyo laser scanner

**Environment:**
- Ubuntu 22.04
- ROS2 Humble
- Gazebo Classic 11
- MuJoCo

## 2. Package Structure

```
youbot_description/
├── launch/
│   ├── gazebo.launch.py
│   └── rviz.launch.py
├── meshes/
├── mujoco/
│   ├── youbot.xml
│   └── view_mujoco.py
├── rviz/
└── urdf/
    └── youbot.urdf
```

## 3. Gazebo Simulation

### 3.1 Launch File

In [ ]:
# gazebo.launch.py

import os
from ament_index_python.packages import get_package_share_directory
from launch import LaunchDescription
from launch.actions import IncludeLaunchDescription, SetEnvironmentVariable
from launch.launch_description_sources import PythonLaunchDescriptionSource
from launch_ros.actions import Node

def generate_launch_description():
    package_name = 'youbot_description'
    pkg_share = get_package_share_directory(package_name)
    
    urdf_file = os.path.join(pkg_share, 'urdf', 'youbot.urdf')
    
    with open(urdf_file, 'r') as f:
        robot_description_content = f.read()
    
    gazebo_model_path = SetEnvironmentVariable(
        'GAZEBO_MODEL_PATH',
        os.path.join(pkg_share, '..') + ':' + os.environ.get('GAZEBO_MODEL_PATH', '')
    )
    
    robot_state_publisher = Node(
        package='robot_state_publisher',
        executable='robot_state_publisher',
        parameters=[{
            'robot_description': robot_description_content,
            'use_sim_time': True
        }]
    )

    gazebo = IncludeLaunchDescription(
        PythonLaunchDescriptionSource([
            os.path.join(get_package_share_directory('gazebo_ros'), 
                        'launch', 'gazebo.launch.py')
        ]),
        launch_arguments={'verbose': 'true'}.items()
    )

    spawn_entity = Node(
        package='gazebo_ros',
        executable='spawn_entity.py',
        arguments=['-topic', 'robot_description', '-entity', 'youbot',
                   '-x', '0.0', '-y', '0.0', '-z', '0.1'],
        output='screen'
    )

    return LaunchDescription([
        gazebo_model_path,
        robot_state_publisher,
        gazebo,
        spawn_entity,
    ])

### 3.2 Joint State Publisher Plugin

In [ ]:
# XML in URDF

joint_state_plugin = """
<gazebo>
  <plugin name="gazebo_ros_joint_state_publisher"
          filename="libgazebo_ros_joint_state_publisher.so">
    <ros>
      <namespace>/youbot</namespace>
    </ros>
    <update_rate>100</update_rate>
    <joint_name>arm_joint_1</joint_name>
    <joint_name>arm_joint_2</joint_name>
    <joint_name>arm_joint_3</joint_name>
    <joint_name>arm_joint_4</joint_name>
    <joint_name>arm_joint_5</joint_name>
    <joint_name>gripper_finger_joint_l</joint_name>
    <joint_name>gripper_finger_joint_r</joint_name>
    <joint_name>wheel_joint_bl</joint_name>
    <joint_name>wheel_joint_br</joint_name>
    <joint_name>wheel_joint_fl</joint_name>
    <joint_name>wheel_joint_fr</joint_name>
  </plugin>
</gazebo>
"""

### 3.3 Planar Move Plugin (Base Controller)

In [ ]:
# XML in URDF

planar_move_plugin = """
<gazebo>
  <plugin name='base_controller' filename='libgazebo_ros_planar_move.so'>
    <ros>
      <namespace>/youbot</namespace>
    </ros>
    <update_rate>100</update_rate>
    <publish_rate>10</publish_rate>
    <publish_odom>true</publish_odom>
    <publish_odom_tf>true</publish_odom_tf>
    <robot_base_frame>base_footprint</robot_base_frame>
  </plugin>
</gazebo>
"""

### 3.4 Laser Scanner Plugin

In [ ]:
# XML in URDF

laser_plugin = """
<gazebo reference="base_laser_front_link">
  <sensor type="ray" name="hokuyo_front">
    <visualize>true</visualize>
    <update_rate>40</update_rate>
    <ray>
      <scan>
        <horizontal>
          <samples>720</samples>
          <min_angle>-1.570796</min_angle>
          <max_angle>1.570796</max_angle>
        </horizontal>
      </scan>
      <range>
        <min>0.10</min>
        <max>10.0</max>
      </range>
    </ray>
    <plugin name="gazebo_ros_head_hokuyo_controller" 
            filename="libgazebo_ros_ray_sensor.so">
      <ros>
        <namespace>/youbot</namespace>
      </ros>
      <output_type>sensor_msgs/LaserScan</output_type>
    </plugin>
  </sensor>
</gazebo>
"""

### 3.5 Available ROS Topics

- `/youbot/cmd_vel` - Velocity commands (geometry_msgs/Twist)
- `/youbot/odom` - Odometry (nav_msgs/Odometry)
- `/youbot/joint_states` - Joint states (sensor_msgs/JointState)
- `/youbot/scan` - Laser scan (sensor_msgs/LaserScan)
- `/robot_description` - URDF
- `/tf` - Transforms

## 4. MuJoCo Simulation

### 4.1 Viewer Script

In [ ]:
# view_mujoco.py

import mujoco
import mujoco.viewer
import os

def main():
    script_dir = os.path.dirname(os.path.abspath(__file__))
    model_path = os.path.join(script_dir, 'youbot.xml')
    
    model = mujoco.MjModel.from_xml_path(model_path)
    data = mujoco.MjData(model)
    
    print(f"Bodies: {model.nbody}")
    print(f"Joints: {model.njnt}")
    print(f"Actuators: {model.nu}")
    
    with mujoco.viewer.launch_passive(model, data) as viewer:
        viewer.cam.azimuth = 135
        viewer.cam.elevation = -20
        viewer.cam.distance = 2.0
        
        while viewer.is_running():
            mujoco.mj_step(model, data)
            viewer.sync()

if __name__ == "__main__":
    main()

### 4.2 Joints

**Wheels:** wheel_joint_fl, wheel_joint_fr, wheel_joint_bl, wheel_joint_br

**Arm:** arm_joint_1 to arm_joint_5

**Gripper:** gripper_finger_joint_l, gripper_finger_joint_r

### 4.3 DRL Environment Template

In [ ]:
import gymnasium as gym
import mujoco
import numpy as np

class YouBotEnv(gym.Env):
    def __init__(self):
        self.model = mujoco.MjModel.from_xml_path('youbot.xml')
        self.data = mujoco.MjData(self.model)
        
        obs_dim = self.model.nq + self.model.nv
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf, shape=(obs_dim,), dtype=np.float64
        )
        self.action_space = gym.spaces.Box(
            low=-1.0, high=1.0, shape=(self.model.nu,), dtype=np.float64
        )
    
    def step(self, action):
        self.data.ctrl[:] = action
        mujoco.mj_step(self.model, self.data)
        obs = np.concatenate([self.data.qpos, self.data.qvel])
        reward = 0.0
        return obs, reward, False, False, {}
    
    def reset(self, seed=None):
        mujoco.mj_resetData(self.model, self.data)
        return np.concatenate([self.data.qpos, self.data.qvel]), {}

## 5. Issues Fixed Today

### Issue 1: gzclient crash (exit -9)

**Cause:** Wayland incompatibility

**Fix:** Set QT_QPA_PLATFORM=xcb

In [ ]:
from launch.actions import SetEnvironmentVariable

SetEnvironmentVariable('QT_QPA_PLATFORM', 'xcb')

### Issue 2: gzserver segfault (exit -11)

**Cause:** Duplicate base_controller plugin in URDF (appeared twice)

**Fix:** Removed duplicate, kept one instance

### Issue 3: p3d plugin error

**Error:** body_name: pen does not exist

**Cause:** Gazebo renames links during URDF-to-SDF conversion

**Fix:** Commented out p3d plugin

In [ ]:
# Disabled in URDF:

p3d_disabled = """
<!-- p3d plugin disabled - causes issues with link name conversion
<gazebo>
  <plugin name="gazebo_ros_p3d" filename="libgazebo_ros_p3d.so">
    <body_name>base_link</body_name>
  </plugin>
</gazebo>
-->
"""

### Issue 4: spawn_entity timeout

**Error:** Service /spawn_entity unavailable

**Cause:** gzserver was crashing before service became available

**Fix:** Resolved by fixing issues 2 and 3

## 6. Commands Reference

### Build Package

In [ ]:
%%bash
cd ~/youbot/src/youbot_description
colcon build --packages-select youbot_description --symlink-install
source install/setup.bash

### Launch Gazebo

In [ ]:
%%bash
source ~/youbot/src/youbot_description/install/setup.bash
ros2 launch youbot_description gazebo.launch.py

### Launch RViz

In [ ]:
%%bash
source ~/youbot/src/youbot_description/install/setup.bash
ros2 launch youbot_description rviz.launch.py

### View MuJoCo Model

In [ ]:
%%bash
cd ~/youbot/src/youbot_description/mujoco
python3 view_mujoco.py

### Kill Gazebo Processes

In [ ]:
%%bash
pkill -9 gzserver
pkill -9 gzclient

### Check ROS Topics

In [ ]:
%%bash
ros2 topic list
ros2 topic echo /youbot/joint_states --once

### Move Robot Base

In [ ]:
%%bash
# Move forward
ros2 topic pub /youbot/cmd_vel geometry_msgs/Twist "{linear: {x: 0.5}, angular: {z: 0.0}}" --once

# Rotate
ros2 topic pub /youbot/cmd_vel geometry_msgs/Twist "{linear: {x: 0.0}, angular: {z: 0.5}}" --once

## 7. Summary

**Working:**
- URDF model
- Gazebo launch
- MuJoCo model
- RViz
- Base control (/youbot/cmd_vel)
- Joint states
- Laser scanner

**Needs work:**
- Arm trajectory controller
- ros2_control integration
- MoveIt2
- Nav2

---

December 13, 2025